# Initializing the System State

## Overview

### Questions

* How do I place particles in the initial condition?
* What units does HOOMD-blue use?

### Objectives

* Describe the basic properties of the **initial condition**, including particle **position**, **orientation**, **type** and the **periodic box**.
* Briefly describe HOOMD-blues system of **units**.
* Demonstrate writing a system to a **GSD** file.
* Show how to initialize **Simulation state** from a **GSD** file.

In [1]:
import hoomd
import math
import itertools
import numpy

## Components of the system state

We need to initialize the system **state** before we can run a simulation.
The **initial condition** describes the the **position** and **orientation** of every particle in the system and the **periodic box** at the start of the simulation.

The hard regular octahedra system self-assembles the the Minkowski structure (TODO cite), which is similar to bcc.
Self-assembly studies usually consist of many thousands of particles, but that is large for a short tutorial so we will use 128 particles, a number commensurate with the *bcc* structure.

In [2]:
N = 250

## Placing particles

In hard particle Monte Carlo, valid particle configurations have no overlaps.
The octahedron particle we are using in this tutorial sits inside a sphere of diameter 1, so let's place particles a little bit further than that apart on a *K*x*K*x*K* simple cubic lattice of width *L*.

In [3]:
spacing = 1.1
K = math.ceil(N**(1/3))
L = K * spacing

In HOOMD, particles **positions** must be placed inside a **period box**.
Cubic boxes range from -*L*/2 to *L*/2.
Use *itertools* and *numpy* to place the positions on the lattice.

In [4]:
x = numpy.arange(-L/2, L/2, spacing)
position = list(itertools.product(x, repeat=3))
print(position[0:4])

[(-3.8500000000000005, -3.8500000000000005, -3.8500000000000005), (-3.8500000000000005, -3.8500000000000005, -2.7500000000000004), (-3.8500000000000005, -3.8500000000000005, -1.6500000000000004), (-3.8500000000000005, -3.8500000000000005, -0.5500000000000003)]


128 is less than `6*6*6`, so we need filter this list down to *N* particles.

In [5]:
position = position[0:N]

We also need to set the **orientation** for each particle.
HOOMD represents orientations with quaternions.
The quaternion (1, 0, 0, 0) represents no rotation, set that for all particles:

In [6]:
orientation = [(1, 0, 0, 0)]*N

## Units

HOOMD-blue does not adopt any particular real system of **units**. 
Instead, HOOMD-blue uses an *internally self-consistent* system of **units** and is compatible with *many* systems of **units**.
For example: if you select the units of *meter*, *Joule*, and *kilogram* for length, energy and mass then the units of force will be *Newtons* and velocity will be *meters/second*.
A popular system of units for nano-scale systems is *nanometers*, *kilojoules/mol*, and *atomic mass units*.

In HPMC, the *primary* **unit** the applies is that of *length*.
Mass exists, but is factored out of the partition function and does not enter into the simulation.
The actual scale of the energy **unit** is irrelevant in athermal HPMC systems where overlapping energies are infinite and valid configurations have zero potential energy.
However, energy does appear implicitly in **derived units** like *pressure* $\left(\frac{\mathrm{energy}}{\mathrm{distance}^3}\right)$.
In HPMC, $k_\mathrm{B}T$ is assumed to be 1 energy unit.
You can convert a pressure value $P$ in HPMC's units to a non-dimensional value with $P^* = \frac{P\sigma^3}{k_\mathrm{B}T}$ where $\sigma$ is a representative length in your system, such as the particle's diameter or edge length.

## Writing the configuration to disk

[**GSD**](https://gsd.readthedocs.io/) files store the **periodic box**, particle **positions**, **orientations**,  and other properties of the state on disk.
We will use the **GSD** Python package to write this file.

In [7]:
import gsd.hoomd

The **Snapshot** object stores the state of the system.

In [8]:
snapshot = gsd.hoomd.Snapshot()
snapshot.particles.N = N
snapshot.particles.position = position
snapshot.particles.orientation = orientation

Each particle also has a **type**.
In HPMC, each **type** has its own shape parameters.
We have a single **type** in this tutorial, so set a **type id** of 0 for all particles:

In [9]:
snapshot.particles.typeid = [0]*N

We also need to set a name for our particle **type**.
Names can be any string.
HOOMD-blue uses the type names to match the parameters you specify in **operations** with the type names present in the **state**.
Give the name `'octahedron'` to the type id 0:

In [10]:
snapshot.particles.types = ['octahedron']

**GSD** represents boxes with a 6-element array. Three box lengths *L_x*, *L_y*, *L_z*, and 3 tilt factors. We have a cubic box here, so the box lengths are equal and the tilt factors are all 0.

In [11]:
snapshot.configuration.box = [L, L, L, 0, 0, 0]

Write this snapshot to `lattice.gsd`:

In [12]:
with gsd.hoomd.open(name='lattice.gsd', mode='wb') as f:
    f.append(snapshot)

## Initializing a Simulation

We can use the file to initialize the **Simulation state**.
First, create a **Simulation**:

In [13]:
cpu = hoomd.device.CPU()
sim = hoomd.Simulation(device=cpu)

HOOMD-blue v2.9.0-1828-gd903f0dc6 DOUBLE HPMC_MIXED SSE SSE2 SSE3 SSE4_1 SSE4_2 AVX AVX2 
Compiled: 06/10/2020
Copyright (c) 2009-2019 The Regents of the University of Michigan.
HOOMD-blue is running on the CPU


The use the `create_state_from_gsd` factory function to read in the GSD file and populate the **state** with the particles and box from the file:

In [14]:
sim.create_state_from_gsd(filename='lattice.gsd')

The next section in this tutorial will show you how to use HPMC to randomize the particle positions and orientations.

[Previous section](02-Performing-Hard-Particle-Monte-Carlo-Simulations.ipynb). [Next section](04-Randomizing-the-System.ipynb).